In [1]:
import os
import google.generativeai as genai
from dotenv import load_dotenv
from prompts import build_prompt

load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

model = genai.GenerativeModel("models/gemini-1.5-flash-latest")


/home/fathimahusna/DR_Agent/dr_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def run_profile_agent(role, experience, skills, rate, tone):
    prompt = build_prompt(role, experience, skills, rate, tone)
    response = model.generate_content(prompt)
    return response.text.strip()

In [3]:
def build_prompt(role, experience, skills, rate, tone):
    return f"""
You are an expert Upwork freelancer.

Create a professional Upwork profile for a {role} with {experience} of experience.
Mention the following core skills: {', '.join(skills)}.
The hourly rate must be included as a string like "{rate}" with a "$" symbol and "/hr".
Write the profile in a {tone.lower()} tone.

Return ONLY a JSON object with these fields:
- title (string)
- overview (string)
- skills (list of strings)
- hourly_rate (string formatted like "$15/hr")
- profile_tips (list of 3–5 strings)

Respond with valid JSON only. No markdown, no explanations.
"""

In [4]:
from pydantic import BaseModel
from typing import List

class UpworkProfile(BaseModel):
    title: str
    overview: str
    skills: List[str]
    hourly_rate: str
    profile_tips: List[str]


In [5]:
import json
import re
from schema import UpworkProfile
from agent import run_profile_agent

def extract_json(text):
    """Extracts the first JSON object from raw Gemini output (removes markdown, etc.)"""
    match = re.search(r"\{.*\}", text, re.DOTALL)
    return match.group(0) if match else None

def main():
    role = "Python Developer"
    experience = "2 years"
    skills = ["Selenium", "Web Scraping", "Automation"]
    rate = "$15/hr",
    tone = "Professional"

    raw_output = run_profile_agent(role, experience, skills, rate, tone)
    cleaned_output = extract_json(raw_output)

    if not cleaned_output:
        print("❌ Failed to extract JSON from output.")
        print("Raw Output:\n", raw_output)
        return

    try:
        parsed = json.loads(cleaned_output)

        # Auto-correct hourly_rate if it's numeric
        if isinstance(parsed.get("hourly_rate"), (int, float)):
            parsed["hourly_rate"] = f"${parsed['hourly_rate']}/hr"

        profile = UpworkProfile(**parsed)
        print("✅ Valid profile generated:\n")
        print(profile.model_dump_json(indent=2))

    except Exception as e:
        print("❌ Validation error:", e)
        print("\nCleaned Output:\n", cleaned_output)


if __name__ == "__main__":
    main()


✅ Valid profile generated:

{
  "title": "Experienced Python Developer | Selenium, Web Scraping & Automation Expert",
  "overview": "Highly proficient Python developer with 2 years of experience in building robust and efficient automation solutions.  My expertise lies in leveraging Selenium for browser automation, crafting sophisticated web scraping tools, and developing custom automation scripts to streamline workflows. I'm adept at working independently and collaboratively, consistently delivering high-quality results that meet and exceed client expectations.  I possess a strong understanding of software development best practices, including version control (Git), testing, and debugging.  I'm eager to contribute my skills and experience to your next project.",
  "skills": [
    "Python",
    "Selenium",
    "Web Scraping",
    "Automation",
    "Beautiful Soup",
    "Scrapy",
    "API Integration",
    "REST APIs",
    "Git",
    "Agile",
    "Testing",
    "Debugging",
    "Problem-